## Day 7: Handy Haversacks ---
You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

light red bags contain 1 bright white bag, 2 muted yellow bags.  
dark orange bags contain 3 bright white bags, 4 muted yellow bags.  
bright white bags contain 1 shiny gold bag.  
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.  
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.  
dark olive bags contain 3 faded blue bags, 4 dotted black bags.  
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.  
faded blue bags contain no other bags.  
dotted black bags contain no other bags.  
These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.  

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

A bright white bag, which can hold your shiny gold bag directly.  
A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.  
A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.  
A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.  
So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.  

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

In [115]:
bag_rules_file = "data/07_bag_rules.txt"

In [122]:
# construct the color rule tree
import re

# to extract information from the lines in data file
def format_line(line) :
    # separation between the bag concerned by the rule and the rule
    this_bag, other_bags = line.strip().split(' contain ')
    this_bag = this_bag[:-5]
    
    # case of empty bag :
    if other_bags == 'no other bags.' : return this_bag, {}
         
    # split for list of contained bags
    other_bags = other_bags.split(', ')
    rules_dic = {}
    for ob in other_bags : 
        # regular expressions to find the color of the bag and the number of bags
        ob = re.findall(r"(.+)\sbag", ob)[0]
        n = int(re.findall(r"[0-9]+", ob)[0])
        bag = re.findall(r"[a-zA-Z].*", ob)[0]
#         store the list of rules as a dictionnary of 
#         key, value <==> colors of bags our current bags need to contain, and number of this color it needs to contain
        rules_dic[bag] = n
    return this_bag, rules_dic

# bag rule class to form the natural tree structure
class bag_rule():
#     init a node (we set the structure when reading the text file)
    def __init__(self, color) :
        self.color = color
        self.children = set()
        self.parents = set()
        
#     returns list of tuples (color, number) of bags this color needs to contain
    def children_colors (self) :
        return [(r[0].color, r[1]) for r in self.children]
    
#     returns the list of ags color that have to contain a bag of the color of this rule
    def parents_colors (self) :
        return [r.color for r in self.parents]
        
with open (bag_rules_file, "r") as bag_rules :
    
#     a dictionnary allowing us to get a bag_rule from a color string
    color_to_rule = {}
#     the colors listed in other rules but without a rule yet
    no_rules_yet = set()
#     the rules we have already done 
    done = set()
    
    # loop on lines
    for line in bag_rules :
#         parse the line, get the bag_rule object
        this_bag, rule_dic = format_line(line)
        this_rule = color_to_rule.get(this_bag, bag_rule(this_bag))
        
#         loop on children to set the bag_rule structures
        for color, n  in rule_dic.items() :
#         set the bag_rule tree structure
            rule = color_to_rule.get(color, bag_rule(color))
            this_rule.children.add((rule,n))
            rule.parents.add(this_rule)
#             store in the dictionnary to find rules easily
            color_to_rule[color] = rule
#             store the colors we have seen but not treated yet
            if color not in done : no_rules_yet.add(color)
            
#         manage our sets
        done.add(this_bag)
        no_rules_yet.discard(this_bag)
#         store in the dictionnary to find rules easily
        color_to_rule[this_bag] = this_rule

print(f'we treated')

In [123]:
# to answer the question :
color_init = 'shiny gold'


to_visit = set(color_to_rule[color_init].parents_colors())
visited = set()

while to_visit : 
    color = to_visit.pop()
    visited.add(color)
    for c in color_to_rule[color].parents_colors():
        if c not in visited : 
            to_visit.add(c)
print(f"the number of bags that can recursively contain a {color_init} bag is : {len(visited)}")

the number of bags that can recursively contain a shiny gold bag is : 300


## Part Two ---
It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

faded blue bags contain 0 other bags.  
dotted black bags contain 0 other bags.  
vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.  
dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.  
So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 1 + 1*7 + 2 + 2*11 = 32 bags!  

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!  

Here's another example:  

shiny gold bags contain 2 dark red bags.  
dark red bags contain 2 dark orange bags.  
dark orange bags contain 2 dark yellow bags.  
dark yellow bags contain 2 dark green bags.  
dark green bags contain 2 dark blue bags.  
dark blue bags contain 2 dark violet bags.  
dark violet bags contain no other bags.  
In this example, a single shiny gold bag must contain 126 other bags.  

How many individual bags are required inside your single shiny gold bag?

In [124]:
# to answer the question :
color_init = 'shiny gold'


to_go_back_up = [color_init]
tot_bags = {}

while to_go_back_up : 
    if to_go_back_up[-1] in tot_bags.keys() :
        to_go_back_up.pop()
        continue
    
    cur_rule = color_to_rule[to_go_back_up[-1]]
    cur_children = cur_rule.children_colors()
    
    all_children_calc = True
    count = 0
    for child_color, child_n in cur_children :
        if child_color not in tot_bags.keys() :
            all_children_calc = False
            to_go_back_up.append(child_color)
        else:
            count += (tot_bags[child_color] + 1) * child_n
    
    if all_children_calc :
        tot_bags[to_go_back_up.pop()] = count
        
print(f"the number of bags that a {color_init} has to contain is : {tot_bags[color_init]}")

the number of bags that a shiny gold has to contain is : 8030
